In [1]:
from xmlrpc.server import SimpleXMLRPCRequestHandler
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCServer
import sqlite3
from datetime import datetime
import pandas as pd
import numpy as np


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 6060),
                            requestHandler=RequestHandler, allow_none=True)

db_connection = sqlite3.connect('log.db')
db_cursor = db_connection.cursor()

is_table_created = False

def store(operation, time, duration):
    
    db_cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS logs
        (
            log_id integer primary key autoincrement,
            operation TEXT,
            operation_time timestamp,
            operation_duration real 
        );
        """
    )
    db_connection.commit()

    log = (operation, str(time), duration)
    db_cursor.execute("INSERT INTO logs(operation, operation_time, operation_duration) VALUES(?, ?, ?);", log)
    db_connection.commit()

server.register_function(store, 'store')

def get_all_logs():
    db_cursor.execute('SELECT * FROM logs')
    return db_cursor.fetchall()

server.register_function(get_all_logs, 'get_all_logs')


def get_logs(operation, time_left, time_right, duration):
    time_left = datetime.strptime(time_left, '%Y-%m-%d %H:%M:%S')
    time_right = datetime.strptime(time_right, '%Y-%m-%d %H:%M:%S')
    time_left = time_left.strftime('%Y%m%dT%H:%M:%S')
    time_right = time_right.strftime('%Y%m%dT%H:%M:%S')
    query = "SELECT * FROM logs"
    wheres = list()
    if operation is not None:
        wheres.append("operation = '" + operation + "'")
    if time_left is not None and time_right is not None:
        wheres.append("operation_time > '" + time_left + "' and operation_time < '" + time_right + "'")
    if duration is not None:
        wheres.append("operation_duration < " + str(duration))
    joined_wheres = " and ".join(wheres)
    db_cursor.execute(query + ' where ' + joined_wheres)
    return db_cursor.fetchall()

server.register_function(get_logs, 'get_logs')

print ("Listening on port 6060...")
server.serve_forever()

Listening on port 6060...


127.0.0.1 - - [12/Dec/2023 15:14:40] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:14:46] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:14:52] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:14:58] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:15:04] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:15:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:15:17] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:15:23] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:15:29] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:15:36] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:15:53] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:16:00] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:16:08] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:16:19] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2023 15:16:25] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/202

KeyboardInterrupt: 